In [19]:
import cv2
import mediapipe as mp
#얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection# 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils# 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

#동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

#model_selection=0 -> 가까운 거리, =1 -> 5m이상 거리 사람, mindetection->0.5 50%이상신뢰도로 얼굴 인지
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue
        # 보기 편하기 위해 이미지를 좌우를 반전하고, BGR 이미지를 RGB로 변환합니다.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # 성능을 향상시키려면 이미지를 작성 여부를 False으로 설정하세요.
        image.flags.writeable = False
        results = face_detection.process(image)

        # 영상에 얼굴 감지 주석 그리기 기본값 : True.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
        #6개의 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분 , 입 중간부분, 오른쪽 귀 , 왼쪽 귀
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                #print(detection)
                
                #특정 위치 가져오기
                keypoints= detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝
                
                h, w, _ = image.shape# height,width,channel: 이미지로부터 세로, 가로 크기 가져오기
                right_eye = (int(right_eye.x*w), int(right_eye.y*h)) # 이미지 내에서 실제 좌표 (x,y)
                left_eye = (int(left_eye.x*w), int(left_eye.y*h))
                nose_tip = (int(nose_tip.x*w), int(nose_tip.y*h))
                #양 눈에 동그라미 그리기
                cv2.circle(image,right_eye,50,(255,0,0),10,cv2.LINE_AA)#파란색
                cv2.circle(image,left_eye,50,(0,255,0),10,cv2.LINE_AA)#초록색
                cv2.circle(image,nose_tip,50,(0,255,255),10,cv2.LINE_AA)#초록색
        flipped_image = cv2.flip(image, 1)
        resized_image = cv2.resize(flipped_image, None, fx=0.5, fy=0.5)
        cv2.imshow('MediaPipe Face Detection', resized_image)
        if cv2.waitKey(1) == ord ('q'):
            break
cap.release()
cv2.destroyAllWindows()